In [1]:
import pandas as pd
from sklearn.cluster import KMeans, SpectralClustering
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pandas as pd
import seaborn as sns
from sklearn.metrics import silhouette_samples, silhouette_score

In [2]:
df=pd.read_csv('norm.csv')

In [3]:
df.columns

Index(['Sno', 'Stride Length (m)', 'Cadence(steps/min)', 'Leg Length (m)',
       'Age(years)', 'Normal sl', 'norm cade', 'scan sl', 'scan caden',
       'classs'],
      dtype='object')

In [4]:
df_req = df[['Stride Length (m)', 'Cadence(steps/min)', 'Leg Length (m)',
       'Age(years)','classs']]
X  = df_req[['Stride Length (m)', 'Cadence(steps/min)', 'Leg Length (m)',
       'Age(years)']]
labels = df.classs.values

In [5]:
from sklearn.model_selection import train_test_split
x_tr,x_tt,y_tr,y_tt = train_test_split(X,labels,test_size=.30)

In [6]:
model = KMeans(n_clusters = 2)

In [7]:
model.fit(x_tr)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [8]:
print("Silhouette Coefficient: %0.3f"
      % silhouette_score(x_tt, y_tt, metric='euclidean'))

Silhouette Coefficient: 0.059


In [48]:
import numpy as np
from numpy.linalg import norm


class Kmeans:
    '''Implementing Kmeans algorithm.'''

    def __init__(self, n_clusters, max_iter=100, random_state=123):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state
        self.c1,self.c2 = [],[]

    def initializ_centroids(self, X):
        np.random.RandomState(self.random_state)
        random_idx = np.random.permutation(X.shape[0])
        centroids = X[random_idx[:self.n_clusters]]
        return centroids

    def compute_centroids(self, X, labels):
        centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            centroids[k, :] = np.mean(X[labels == k, :], axis=0)
        return centroids

    def compute_distance(self, X, centroids):
        distance = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            row_norm = norm(X - centroids[k, :], axis=1)
            distance[:, k] = np.square(row_norm)
        return distance

    def find_closest_cluster(self, distance):
        return np.argmin(distance, axis=1)

    def compute_sse(self, X, labels, centroids):
        distance = np.zeros(X.shape[0])
        for k in range(self.n_clusters):
            distance[labels == k] = norm(X[labels == k] - centroids[k], axis=1)
        return np.sum(np.square(distance))
    
    def fit(self, X):
        self.centroids = self.initializ_centroids(X)
        for i in range(self.max_iter):
            old_centroids = self.centroids
            distance = self.compute_distance(X, old_centroids)
            self.labels = self.find_closest_cluster(distance)
            self.centroids = self.compute_centroids(X, self.labels)
            if np.all(old_centroids == self.centroids):
                break
        self.error = self.compute_sse(X, self.labels, self.centroids)
    def return_centroids(self):
        return self.centroids
    def predict(self, X):
        distance = self.compute_distance(X, self.centroids)
        return self.find_closest_cluster(distance)
    def clusters(self,X):
        d = self.predict(X)
        for i in range(len(d)):
            if d[i] == 0:
                self.c1.append(i)
            else:
                self.c2.append(i)
        return self.c1,self.c2

In [49]:
a = Kmeans(n_clusters = 2)

In [50]:
b = a.fit(x_tr.values)

In [51]:
v = a.predict(x_tr.values)

In [52]:
print("Silhouette Coefficient: %0.3f"
      % silhouette_score(v.reshape(-1,1), y_tr.reshape(-1,1), metric='euclidean'))

Silhouette Coefficient: 0.103


C:\Users\Lenovo\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
c1,c2 = a.clusters(X)

In [57]:
d = a.return_centroids()